In [1]:
import pandas as pd
from typing import Tuple

In [2]:
CONTRACT_INTRADAY_SLIDING_OPEN_FILE_PATH = '../data/processed/futures_contracts/contract_open_enriched_sliding_open.csv'
CONTRACT_INTRADAY_TRUE_OPEN_FILE_PATH = '../data/processed/futures_contracts/contract_open_enriched_true_open.csv'
CONTRACT_OVERNIGHT_CHANGES_SLIDING_OPEN_FILE_PATH = '../data/processed/futures_contracts/overnight_changes_by_contract_sliding_open.csv'
CONTRACT_OVERNIGHT_CHANGES_TRUE_OPEN_FILE_PATH = '../data/processed/futures_contracts/overnight_changes_by_contract_true_open.csv'

In [3]:
def intraday_open_csv_to_df(filename) -> pd.DataFrame:
    csv_as_df = pd.read_csv(
        filename,
        parse_dates=['DateTime'], 
        usecols=[
          'Symbol','DateTime','Open Minutes Offset','Open','High','Low','Close',
            'Volume','Price Change From Intraday Open','Expiration Date','DTE'
          ]
    )
    return csv_as_df

In [4]:
def overnight_changes_csv_to_df(filename) -> pd.DataFrame:
  csv_as_df = pd.read_csv(
    filename,
    parse_dates=['Date'],
    usecols=['Symbol','Date','12:59 Change','13:04 Change','Last Bar Change']
  )
  return csv_as_df

In [5]:
intraday_sliding_open_df = intraday_open_csv_to_df(CONTRACT_INTRADAY_SLIDING_OPEN_FILE_PATH)
intraday_true_open_df = intraday_open_csv_to_df(CONTRACT_INTRADAY_TRUE_OPEN_FILE_PATH)
overnight_sliding_open_df = overnight_changes_csv_to_df(CONTRACT_OVERNIGHT_CHANGES_SLIDING_OPEN_FILE_PATH)
overnight_true_open_df = overnight_changes_csv_to_df(CONTRACT_OVERNIGHT_CHANGES_TRUE_OPEN_FILE_PATH)

In [6]:
true_open_twelve_fifty_nine_all_bars_positive_change_df = pd.DataFrame()
true_open_twelve_fifty_nine_all_bars_negative_change_df = pd.DataFrame()
true_open_thirteen_oh_four_all_bars_positive_change_df = pd.DataFrame()
true_open_thirteen_oh_four_all_bars_negative_change_df = pd.DataFrame()
true_open_last_bar_all_bars_positive_change_df = pd.DataFrame()
true_open_last_bar_all_bars_negative_change_df = pd.DataFrame()

In [7]:
unique_symbols = list(intraday_true_open_df.Symbol.unique())

In [8]:
overnight_true_open_twelve_fifty_nine_df = overnight_true_open_df[overnight_true_open_df['12:59 Change'].notna()]
overnight_true_open_twelve_fifty_nine_df

,Symbol,Date,12:59 Change,13:04 Change,Last Bar Change
134,LEG09,2008-08-05,-0.500,NaN,-0.425
143,LEG09,2008-08-18,0.050,NaN,0.000
144,LEG09,2008-08-19,-0.575,NaN,-0.150
152,LEG09,2008-08-29,-0.225,NaN,0.300
156,LEG09,2008-09-05,-0.475,-0.475,-0.325
...,...,...,...,...,...
25816,LEZ20,2020-12-14,-0.175,NaN,0.025
25820,LEZ20,2020-12-18,0.550,0.600,0.125
25822,LEZ20,2020-12-22,-0.175,NaN,0.050
25825,LEZ20,2020-12-28,-0.150,-0.150,-0.150


In [9]:
overnight_true_open_twelve_fifty_nine_positive_change_df = overnight_true_open_twelve_fifty_nine_df[overnight_true_open_twelve_fifty_nine_df['12:59 Change'] >= 0]
overnight_true_open_twelve_fifty_nine_negative_change_df = overnight_true_open_twelve_fifty_nine_df[overnight_true_open_twelve_fifty_nine_df['12:59 Change'] < 0]
display(overnight_true_open_twelve_fifty_nine_positive_change_df)
display(overnight_true_open_twelve_fifty_nine_negative_change_df)

,Symbol,Date,12:59 Change,13:04 Change,Last Bar Change
143,LEG09,2008-08-18,0.050,NaN,0.000
165,LEG09,2008-09-18,0.675,0.550,0.250
166,LEG09,2008-09-19,1.625,1.750,0.750
167,LEG09,2008-09-22,0.475,0.425,0.300
169,LEG09,2008-09-24,0.225,0.325,0.150
...,...,...,...,...,...
25804,LEZ20,2020-11-25,0.100,0.175,0.175
25809,LEZ20,2020-12-03,0.050,-0.050,0.175
25810,LEZ20,2020-12-04,0.325,0.400,0.000
25820,LEZ20,2020-12-18,0.550,0.600,0.125


,Symbol,Date,12:59 Change,13:04 Change,Last Bar Change
134,LEG09,2008-08-05,-0.500,NaN,-0.425
144,LEG09,2008-08-19,-0.575,NaN,-0.150
152,LEG09,2008-08-29,-0.225,NaN,0.300
156,LEG09,2008-09-05,-0.475,-0.475,-0.325
162,LEG09,2008-09-15,-0.150,NaN,-0.600
...,...,...,...,...,...
25814,LEZ20,2020-12-10,-0.250,-0.250,-0.225
25815,LEZ20,2020-12-11,-0.200,-0.200,0.000
25816,LEZ20,2020-12-14,-0.175,NaN,0.025
25822,LEZ20,2020-12-22,-0.175,NaN,0.050


In [10]:
def split_intraday_activity_by_overnight_change_for_symbol(
  contract_symbol: str,
  overnight_positive_change_from_close_df: pd.DataFrame,
  overnight_negative_change_from_close_df: pd.DataFrame,
  intraday_df: pd.DataFrame
  ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    '''
    Split the intraday minutes for a given contract symbol into those days minutes which correspond to a positive overnight close change and those days minutes that correspond to a negative overnight close change.
    Return the split data as a tuple of dataframes accordingly
    '''
    # Get a series of dates representing the days where there was a positive change from the prior days close bar for this symbol
    dates_of_positive_change_series = overnight_positive_change_from_close_df[overnight_positive_change_from_close_df['Symbol'] == contract_symbol]['Date'].dt.date
    # Get a series of dates representing the days where there was a negative change from the prior days close bar for this symbol
    dates_of_negative_change_series = overnight_negative_change_from_close_df[overnight_negative_change_from_close_df['Symbol'] == contract_symbol]['Date'].dt.date
    # Filter down our intraday enriched contract data to only include those items matching the the symbol we are currently analyzing
    intraday_for_symbol_df = intraday_df[intraday_df['Symbol'] == contract_symbol]
    # Filter down the rows in the intraday data for this symbol to only those associated with a day where there was a negative overnight change for this type of close
    intraday_minutes_negative_change_df = intraday_for_symbol_df[intraday_for_symbol_df['DateTime'].dt.date.isin(dates_of_negative_change_series)]
    # Filter down the rows in the intraday data for this symbol to only those associated with a day where there was a positive overnight change for this type of close
    intraday_minutes_positive_change_df = intraday_for_symbol_df[intraday_for_symbol_df['DateTime'].dt.date.isin(dates_of_positive_change_series)]
    return (intraday_minutes_positive_change_df, intraday_minutes_negative_change_df)

In [14]:
for symbol in unique_symbols:
  print(f"Splitting intraday minutes for symbol {symbol}")
  intraday_minutes_positive_change_df, intraday_minutes_negative_change_df = split_intraday_activity_by_overnight_change_for_symbol(
    contract_symbol=symbol,
    overnight_positive_change_from_close_df=overnight_true_open_twelve_fifty_nine_positive_change_df,
    overnight_negative_change_from_close_df=overnight_true_open_twelve_fifty_nine_negative_change_df,
    intraday_df=intraday_true_open_df
    )
  true_open_twelve_fifty_nine_all_bars_positive_change_df = pd.concat([true_open_twelve_fifty_nine_all_bars_positive_change_df, intraday_minutes_positive_change_df])
  true_open_twelve_fifty_nine_all_bars_negative_change_df = pd.concat([true_open_twelve_fifty_nine_all_bars_negative_change_df, intraday_minutes_negative_change_df])

Splitting intraday minutes for symbol LEG09
Splitting intraday minutes for symbol LEG10
Splitting intraday minutes for symbol LEG11
Splitting intraday minutes for symbol LEG12
Splitting intraday minutes for symbol LEG13
Splitting intraday minutes for symbol LEG14
Splitting intraday minutes for symbol LEG15
Splitting intraday minutes for symbol LEG16
Splitting intraday minutes for symbol LEG17
Splitting intraday minutes for symbol LEG18
Splitting intraday minutes for symbol LEG19
Splitting intraday minutes for symbol LEG20
Splitting intraday minutes for symbol LEJ08
Splitting intraday minutes for symbol LEJ09
Splitting intraday minutes for symbol LEJ10
Splitting intraday minutes for symbol LEJ11
Splitting intraday minutes for symbol LEJ12
Splitting intraday minutes for symbol LEJ13
Splitting intraday minutes for symbol LEJ14
Splitting intraday minutes for symbol LEJ15
Splitting intraday minutes for symbol LEJ16
Splitting intraday minutes for symbol LEJ17
Splitting intraday minutes for s

In [ ]:
first_index = pd.Timestamp(2008, 8, 5).date()
second_index = pd.Timestamp(2008, 8, 19).date()
temp_series = pd.Series([first_index, second_index])
true_open_minute_bars_df[true_open_minute_bars_df['DateTime'].dt.date.isin(temp_series)]